In [1]:
import pandas as pd
from haversine import haversine
import numpy as np
import datetime
import os
import json
import zipcodes
from fuzzywuzzy import fuzz
os.getcwd()

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


'/home/jian/Projects/Umpqua_Bank/May_2019'

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))

input_address_2019May=pd.read_excel("/home/jian/Projects/Umpqua_Bank/May_2019/store_list/Umpqua Bank Store List 3 4 19.xlsx",dtype=str)
cols_list=input_address_2019May.columns.tolist()
store_address_2018Dec=pd.read_excel("/home/jian/Projects/Umpqua_Bank/May_2019/store_list/UMPQUA - All Locations_old_with_latlng.xlsx",dtype=str)



In [3]:
store_address_2018Dec=store_address_2018Dec[['Store Number','Address','City','State','State Name ',
                                             'Zip Code','Latitude','Longitude','Geo Accuracy','County','Country',
                                             'Country Code','DMA', 'Census Region ']]
store_address_2018Dec['address_str']=store_address_2018Dec['Address']+", "+store_address_2018Dec['City']+", "+store_address_2018Dec['State']+\
                                    ", "+store_address_2018Dec['Zip Code']
store_address_2018Dec['Zip Code'].apply(lambda x: len(x)).unique()

store_address_2018Dec=store_address_2018Dec[['address_str','Store Number','Latitude','Longitude']]
store_address_2018Dec['Latitude']=store_address_2018Dec['Latitude'].astype(float)
store_address_2018Dec['Longitude']=store_address_2018Dec['Longitude'].astype(float)

store_address_2018Dec['Source']='Dec_2018_file'

input_address_2019May=input_address_2019May.reset_index()
store_address_2018Dec=store_address_2018Dec.reset_index()

del input_address_2019May['index']
del store_address_2018Dec['index']

In [4]:
# No need to find the top 5, only the best match works good enough

def find_match_stroes_from_Dec(address_str):
    output_top_5_list=[]
    df_store=pd.DataFrame()
    for i in range(len(store_address_2018Dec)):
        store_num_dec=store_address_2018Dec['Store Number'][i]
        store_add_dec=store_address_2018Dec['address_str'][i]
        ratio=fuzz.ratio(address_str,store_add_dec)

        df=pd.DataFrame({"store_num_dec":store_num_dec,"ratio":ratio,"store_add_dec":store_add_dec},index=[0])
        df_store=df_store.append(df)
    df_store=df_store.sort_values("ratio",ascending=False)
    df_store=df_store.head(1)
    df_store=df_store.reset_index()
    del df_store['index']

    output=str(df_store['store_num_dec'][0])+"|"+str(df_store['ratio'][0])+"|"+str(df_store['store_add_dec'][0])

    return output


In [5]:
input_address_2019May_0=input_address_2019May.copy()
input_address_2019May_0['best_match']=input_address_2019May_0['Address'].apply(lambda x: find_match_stroes_from_Dec(x))


In [6]:
input_address_2019May_0['best_ratio']=input_address_2019May_0['best_match'].apply(lambda x: int(x.split("|")[1]))
input_address_2019May_0['best_add_dec']=input_address_2019May_0['best_match'].apply(lambda x: x.split("|")[2])

In [7]:
input_address_2019May_0[input_address_2019May_0['best_ratio']<85].to_csv("./ratio_85_lower.csv",index=False) # 
list_no_good_match=['50211', '50414','50008']

# 50301 - WA0095 seem to be same location, googled close enough, different in "Ave." and "ST"
# WA0145 1 store in Dec matched to 2 stores in May same address (only suit different)
input_address_2019May_0[input_address_2019May_0['best_ratio']>=85].to_csv("./ratio_85_higher.csv",index=False) # All good checked by eye

In [8]:
input_address_2019May_0=input_address_2019May_0[~input_address_2019May_0['Cost Center'].isin(list_no_good_match)]

del input_address_2019May_0['best_ratio']
del input_address_2019May_0['best_add_dec']

input_address_2019May_0['Store Number']=input_address_2019May_0['best_match'].apply(lambda x: x.split("|")[0])
del input_address_2019May_0['best_match']

# manually match
input_address_2019May_50008=input_address_2019May[input_address_2019May['Cost Center']=="50008"]
input_address_2019May_50008['Store Number']="OR0050"
input_address_2019May_0=input_address_2019May_0.append(input_address_2019May_50008)



input_address_2019May_1=input_address_2019May[input_address_2019May['Cost Center'].isin(list_no_good_match)]
input_address_2019May_1=input_address_2019May_1[input_address_2019May_1['Cost Center']!="50008"]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
input_address_2019May_0=pd.merge(input_address_2019May_0,store_address_2018Dec,on="Store Number",how="left")
input_address_2019May_0=input_address_2019May_0.rename(columns={"address_str":"address_Dec"})


In [10]:
input_address_2019May_1

,Cost Center,Site Description,Phone and Fax,Hours,Drive Up Hours,Address,City,State,Manager
144,50211,Portland - Alberta,M: 503-219-6160\r\nD: 503-219-6161\r\nF: 503-2...,M-Th: 9:00-5:00\r\nF: 9:00-6:00\r\nS:,M-S:,"1745 NE Alberta St., Portland, OR, 97211",Portland,OR,Juli Wright
147,50414,Portland - Fox Tower,M: 971-544-3700\r\nD: 971-544-3701\r\nF: 503-2...,M-Th: 9:00-5:00\r\nF: 9:00-6:00\r\nS: CLOSED,M-S:,"750 SW Yamhill Street, Portland, OR, 97205",Portland,OR,Derrick Luiz


In [11]:
google_add_dict={}
google_add_dict.update({"50211":{'address_google':"1745 NE Alberta St, Portland, OR 97211",'Latitude':45.5592252,'Longitude':-122.6472587,"Store Number":np.nan,"Source":"Google_Map"}})
google_add_dict.update({"50414":{'address_google':"750 SW Yamhill St, Portland, OR 97205",'Latitude':45.5187616,'Longitude':-122.6810077,"Store Number":np.nan,"Source":"Google_Map"}})


input_address_2019May_1['Store Number']=np.nan
input_address_2019May_1['address_google']=input_address_2019May_1['Cost Center'].apply(lambda x: google_add_dict[x]['address_google'])
input_address_2019May_1['Latitude']=input_address_2019May_1['Cost Center'].apply(lambda x: google_add_dict[x]['Latitude'])
input_address_2019May_1['Longitude']=input_address_2019May_1['Cost Center'].apply(lambda x: google_add_dict[x]['Longitude'])
input_address_2019May_1['Source']=input_address_2019May_1['Cost Center'].apply(lambda x: google_add_dict[x]['Source'])


In [12]:
output_store=input_address_2019May_0.append(input_address_2019May_1)
print(output_store.shape)
print(output_store['Cost Center'].nunique())
print(output_store['Store Number'].nunique())
# 2 nan, 1 store matched to 2 addresses in May

(247, 15)
247
244


In [13]:
output_store=output_store.reset_index()
del output_store['index']

In [14]:
print(output_store['Latitude'].apply(lambda x: type(x)).unique())
print(output_store['Longitude'].apply(lambda x: type(x)).unique())

[<class 'float'>]
[<class 'float'>]


In [15]:
output_store.head(2)

,Address,City,Cost Center,Drive Up Hours,Hours,Latitude,Longitude,Manager,Phone and Fax,Site Description,Source,State,Store Number,address_Dec,address_google
0,"333 SW Ellsworth Street, Albany, OR, 97321",Albany,50030,M-F: 8:30-5:30\r\nS: 10:00-2:00,M-F: 9:00-5:00\r\nS: CLOSED,44.63516,-123.105255,Jack Mochnick,M: 541-967-7000\r\nD: 541-967-4214\r\nF: 541-8...,Albany,Dec_2018_file,OR,OR0113,"333 SW Ellsworth Street, Albany, OR, 97321",NaN
1,"7921 Walerga Road, Antelope, CA, 95843",Antelope,50149,nan,M-Th: 9:00-5:00\r\nF: 9:00-6:00\r\nS: 9:00-1:00,38.71305,-121.365054,Dominic Mecklenburg,M: 916-745-7100\r\nD: 916-745-7101,Antelope - Crossing,Dec_2018_file,CA,CA0071,"7921 Walerga Road, Antelope, CA, 95843",NaN


In [16]:
output_store['zip_list_0_3']=np.nan
output_store['zip_list_4_6']=np.nan

df_output_by_zip=pd.DataFrame()
for index,row in output_store.iterrows():
    location_id_May=row['Cost Center']
    location_id_Dec=row['Store Number']
    
    list_0_to_3=[]
    list_4_to_6=[]
    
    store_center=(row['Latitude'],row['Longitude'])
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=6:
            df=pd.DataFrame({"location_id_May":location_id_May,'location_id_Dec':location_id_Dec,"zip_cd":zip_cd,"distance_miles":dist},index=[0])
            df_output_by_zip=df_output_by_zip.append(df)
            
            if dist<=3:
                list_0_to_3=list_0_to_3+[zip_cd]
            else:
                list_4_to_6=list_4_to_6+[zip_cd]
    
    output_store.loc[index,'zip_list_0_3']=str(list_0_to_3)
    output_store.loc[index,'zip_list_4_6']=str(list_4_to_6)
            


In [17]:
output_store=output_store[cols_list+[x for x in output_store.columns.tolist() if x not in cols_list]]
output_store['store_zip']=output_store['Address'].apply(lambda x: x.split(", ")[len(x.split(", "))-1])
output_store['store_zip']=output_store['store_zip'].apply(lambda x: x.split("-")[0])
output_store['Store Number']=output_store['Store Number'].fillna("nan")

In [18]:
def func_set(x):
    return set(x)

df_zips_with_store=output_store.groupby('store_zip')['Cost Center',"Store Number"].agg(func_set).reset_index()


df_output_by_zip_by_stores=df_output_by_zip[['zip_cd','location_id_May','location_id_Dec','distance_miles']]


In [19]:
zip_nielsen=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2,6,9],skiprows=1)
zip_nielsen.columns=['zip_cd','DMA','ST','CTY']
zip_nielsen=zip_nielsen.drop_duplicates()
zip_nielsen=zip_nielsen.groupby(['zip_cd'])['DMA','ST','CTY'].agg(func_set).reset_index()
zip_nielsen.head(2)


,zip_cd,DMA,ST,CTY
0,00501,{NEW YORK},{NY},{SUFFOLK}
1,00544,{NEW YORK},{NY},{SUFFOLK}


In [20]:
df_nearset_store_by_zip=df_output_by_zip.sort_values(['zip_cd','distance_miles']).drop_duplicates("zip_cd")
df_nearset_store_by_zip=df_nearset_store_by_zip.rename(columns={"location_id_Dec":"nearest_store_number","location_id_May":"nearest_cost_center"})
df_nearset_store_by_zip=pd.merge(df_nearset_store_by_zip,zip_nielsen,on="zip_cd",how="left")
df_nearset_store_by_zip=pd.merge(df_nearset_store_by_zip,df_zips_with_store,left_on="zip_cd",right_on="store_zip",how="left")

df_nearset_store_by_zip=df_nearset_store_by_zip[['zip_cd','nearest_cost_center','nearest_store_number','distance_miles','Cost Center','Store Number','ST','CTY','DMA']]
df_nearset_store_by_zip['city']=np.nan
for index,row in df_nearset_store_by_zip.iterrows():
    zip_cd=row['zip_cd']
    city_list=[]
    try:
        list_zip_info=zipcodes.matching(zip_cd)
        for i in range(len(list_zip_info)):
            city_list=city_list+[list_zip_info[i]['city']]
    except:
        city_list=[]
    city_set=set(city_list)
    
    df_nearset_store_by_zip.loc[index,'city']=str(city_set)
            
df_nearset_store_by_zip=df_nearset_store_by_zip.sort_values('zip_cd')

In [21]:
mapping_zip_info=df_nearset_store_by_zip[['zip_cd','Cost Center','Store Number','ST','CTY','DMA','city']]

df_output_by_zip_by_stores=pd.merge(df_output_by_zip_by_stores,mapping_zip_info,on="zip_cd",how="left")

In [22]:
df_output_by_zip_by_stores_0_3=df_output_by_zip_by_stores[df_output_by_zip_by_stores['distance_miles']<=3]
df_output_by_zip_by_stores_4_6=df_output_by_zip_by_stores[df_output_by_zip_by_stores['distance_miles']>3]

In [32]:
writer=pd.ExcelWriter("/home/jian/Projects/Umpqua_Bank/May_2019/output/Umpqua_zip_list_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_nearset_store_by_zip.index=np.where(df_nearset_store_by_zip['distance_miles']<=3,"0_to_3_miles","4_to_6_miles")

df_nearset_store_by_zip.to_excel(writer,"all_unique_zips",index=True)
df_output_by_zip_by_stores_0_3.to_excel(writer,"zip_0_3_by_store",index=False)
df_output_by_zip_by_stores_4_6.to_excel(writer,"zip_4_6_by_store",index=False)
df_output_by_zip_by_stores.to_excel(writer,"zip_all_by_stores",index=False)
output_store.to_excel(writer,"store_list",index=False)



In [33]:
all_zips_0_3=df_nearset_store_by_zip[df_nearset_store_by_zip['distance_miles']<=3]['zip_cd'].tolist()

In [34]:
for state,group in output_store.groupby(['State']):
    group['zip_list_0_3']=group['zip_list_0_3'].apply(lambda x: eval(x))
    group['zip_list_4_6']=group['zip_list_4_6'].apply(lambda x: eval(x))
    
    zips_state_0_3=[]
    zips_state_4_6=[]
    
    for index,row in group.iterrows():
        zips_state_0_3=zips_state_0_3+row['zip_list_0_3']
        zips_state_4_6=zips_state_4_6+row['zip_list_4_6']
        
    zips_state_0_3=list(set(zips_state_0_3))
    zips_state_4_6=list(set(zips_state_4_6))
    zips_state_4_6=[x for x in zips_state_4_6 if x not in all_zips_0_3]
    
    if len(zips_state_0_3)>=len(zips_state_4_6):
        zips_state_4_6=zips_state_4_6+[np.nan]*(len(zips_state_0_3)-len(zips_state_4_6))
    else:
        zips_state_0_3=zips_state_0_3+[np.nan]*(len(zips_state_4_6)-len(zips_state_0_3))
        
    df=pd.DataFrame({"Store_State":state,'zips_0_3':zips_state_0_3,'zips_4_6':zips_state_4_6},index=[x for x in range(len(zips_state_0_3))])
    df.to_excel(writer,state,index=False)

        
writer.save()    
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:

writer.save()